In [1]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots
import src.data.helpers as dh
import src.models.helpers as mh
import plotly.graph_objects as go
import logging

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)


import pandas as pd
import numpy as np

### Run this to bypass the two next sections (get unblocked fev1 and preprocessd data)

In [2]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx_and_heighest_obs_per_id")

# Get unblocked FEV1

In [ ]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")

In [ ]:
df = df.drop(columns=["PEF", "ecPEF (L/s)", "PEF (L/s)"])
df = df.dropna(subset=["FEV1", "O2 Saturation", "FEF2575"])
df["ecFEF2575%ecFEV1"] = df["FEF2575"] / df["FEV1"] * 100
print(df.shape)
print("IDs: ", df["ID"].nunique())
df.head()

In [ ]:
# Add unblocked FEV1
def get_unblocked_fev1_for_ID(df):
    # Find index where ecFEV1 is max
    idx = df["ecFEV1"].idxmax()
    df["Max ecFEV1"] = df.loc[idx, "ecFEV1"]
    df["Max ecFEF2575"] = df.loc[idx, "ecFEF2575"]
    return df


df = (
    df.groupby("ID")
    .apply(get_unblocked_fev1_for_ID)
    .drop(columns=["ID"])
    .reset_index()
    .drop(columns=["level_1"])
)

In [ ]:
# How often is max ecFEV1 obtained at the same time as max ecFEF2575?
# Filter idx where max ecFEV1 = ecFEV1
idx_max = df["Max ecFEV1"] == df["ecFEV1"]
df_tmp = df[idx_max].copy()
df_tmp["Drop from max ecFEF2575"] = np.abs(
    df_tmp["Max ecFEF2575"] - df_tmp["ecFEF2575"]
)

(df_tmp["Drop from max ecFEF2575"] == 0).value_counts()
# Most of the time, max ecFEV1 is obtained at the same time as max ecFEF2575

# Preprocess breathe Data

In [ ]:
(HFEV1, ecFEV1, AR, HO2Sat, O2SatFFA, IA, UO2Sat, O2Sat, ecFEF2575prctecFEV1) = (
    var_builders.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(
        160, 40, "Male"
    )
)

In [ ]:
# Applied get_bin_for_value to all inputs and updated excel file
df[f"idx {ecFEV1.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["ecFEV1"], ecFEV1)[1], axis=1
)
df[f"idx {ecFEF2575prctecFEV1.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["ecFEF2575%ecFEV1"], ecFEF2575prctecFEV1)[1],
    axis=1,
)
df[f"idx {O2Sat.name}"] = df.apply(
    lambda row: ih.get_bin_for_value(row["O2 Saturation"], O2Sat)[1], axis=1
)

# Use approximate inference (slicing)

In [94]:
# Plot function
def plot_one_and_two_days_models_res(
    df_query_res_one_day,
    df_query_res_one_day_fef,
    df_query_res_two_days,
    df_query_res_two_days_fef,
    HFEV1,
    HO2Sat,
    AR,
    IA,
    df,
    save=False,
):

    lay_row = [{}, {}, {}, {}]
    lay_full_row = [{"colspan": 4}, None, None, None]
    # 4 rows: prior, posterior for healthy var, then ar day 1, ar day 3
    fig = make_subplots(
        rows=8,
        cols=4,
        specs=[
            lay_row,
            lay_row,
            lay_row,
            lay_row,
            lay_row,
            lay_row,
            lay_full_row,
            lay_full_row,
        ],
    )

    # Day
    title_day = f"ecFEV1={df_query_res_two_days_fef.loc[0, 'ecFEV1']:.2g}, O2 Saturation={df_query_res_two_days_fef.loc[0, 'O2 Saturation']:.2g}%, ecFEF2575%ecFEV1={df_query_res_two_days_fef.loc[0, 'ecFEF2575%ecFEV1']:.2f}%"
    # Max
    title_max_FEV1_day = f"ecFEV1={df_query_res_two_days_fef.loc[1, 'ecFEV1']:.2g}, O2 Saturation={df_query_res_two_days_fef.loc[1, 'O2 Saturation']:.2g}%, ecFEF2575%ecFEV1={df_query_res_two_days_fef.loc[1, 'ecFEF2575%ecFEV1']:.2f}%"

    title_prefix = f"Comparing one vs two days model ID={df_query_res_two_days_fef.loc[0, 'ID']}, {df_query_res_two_days_fef.loc[0, 'Age']}y, {df_query_res_two_days_fef.loc[0, 'Height']}cm, {df_query_res_two_days_fef.loc[0, 'Sex']}"

    # One day
    hfev1_one_day = df_query_res_one_day.loc[0, HFEV1.name]
    ho2sat_one_day = df_query_res_one_day.loc[0, HO2Sat.name]
    ar_one_day = df_query_res_one_day.loc[0, AR.name]
    ia_one_day = df_query_res_one_day.loc[0, IA.name]

    # One day fef
    hfev1_one_day_fef = df_query_res_one_day_fef.loc[0, HFEV1.name]
    ho2sat_one_day_fef = df_query_res_one_day_fef.loc[0, HO2Sat.name]
    ar_one_day_fef = df_query_res_one_day_fef.loc[0, AR.name]
    ia_one_day_fef = df_query_res_one_day_fef.loc[0, IA.name]

    # Two days
    ar_day_1 = df_query_res_two_days.loc[0, AR.name]
    ar_max = df_query_res_two_days.loc[1, AR.name]
    ia_day_1 = df_query_res_two_days.loc[0, IA.name]
    ia_max = df_query_res_two_days.loc[1, IA.name]
    hfev1_two_days = df_query_res_two_days.loc[0, HFEV1.name]
    ho2sat_two_days = df_query_res_two_days.loc[0, HO2Sat.name]

    # Two days fef
    ar_day_1_fef = df_query_res_two_days_fef.loc[0, AR.name]
    ar_max_fef = df_query_res_two_days_fef.loc[1, AR.name]
    ia_day_1_fef = df_query_res_two_days_fef.loc[0, IA.name]
    ia_max_fef = df_query_res_two_days_fef.loc[1, IA.name]
    hfev1_two_days_fef = df_query_res_two_days_fef.loc[0, HFEV1.name]
    ho2sat_two_days_fef = df_query_res_two_days_fef.loc[0, HO2Sat.name]

    # Priors
    ih.plot_histogram(
        fig, HFEV1, HFEV1.cpt, HFEV1.a, HFEV1.b, 1, 1, HFEV1.name + " prior", "#009e73"
    )
    ih.plot_histogram(
        fig,
        HO2Sat,
        HO2Sat.cpt,
        HO2Sat.a,
        HO2Sat.b,
        1,
        2,
        HO2Sat.name + " prior",
        "#0072b2",
    )
    # Posteriors
    # One days HFEV1
    ih.plot_histogram(
        fig,
        HFEV1,
        hfev1_one_day,
        HFEV1.a,
        HFEV1.b,
        2,
        1,
        "HFEV1" + " one day",
        "#009e73",
    )
    ih.plot_histogram(
        fig,
        HFEV1,
        hfev1_one_day_fef,
        HFEV1.a,
        HFEV1.b,
        2,
        3,
        "HFEV1" + " one day with FEF25-75",
        "#009e73",
    )
    # One days HO2Sat
    ih.plot_histogram(
        fig,
        HO2Sat,
        ho2sat_one_day,
        HO2Sat.a,
        HO2Sat.b,
        2,
        2,
        "HO2Sat" + " one day",
        "#0072b2",
    )
    ih.plot_histogram(
        fig,
        HO2Sat,
        ho2sat_one_day_fef,
        HO2Sat.a,
        HO2Sat.b,
        2,
        4,
        "HO2Sat" + " one day with FEF25-75",
        "#0072b2",
    )
    # One days AR
    ih.plot_histogram(
        fig,
        AR,
        ar_one_day,
        AR.a,
        AR.b,
        3,
        1,
        "AR" + " one day",
        colour="#d55e00",
    )
    ih.plot_histogram(
        fig,
        AR,
        ar_one_day_fef,
        AR.a,
        AR.b,
        3,
        3,
        "AR" + " one day with FEF25-75",
        colour="#d55e00",
    )
    # One days IA
    ih.plot_histogram(
        fig,
        IA,
        ia_one_day,
        IA.a,
        IA.b,
        3,
        2,
        IA.name + " one day",
        "#cc79a7",
    )
    ih.plot_histogram(
        fig,
        IA,
        ia_one_day_fef,
        IA.a,
        IA.b,
        3,
        4,
        IA.name + " one day with FEF25-75",
        "#cc79a7",
    )
    # Two days HFEV1
    ih.plot_histogram(
        fig,
        HFEV1,
        hfev1_two_days,
        HFEV1.a,
        HFEV1.b,
        4,
        1,
        "HFEV1" + " shared posterior",
        "#009e73",
    )
    ih.plot_histogram(
        fig,
        HFEV1,
        hfev1_two_days_fef,
        HFEV1.a,
        HFEV1.b,
        4,
        3,
        "HFEV1" + " shared posterior with FEF25-75",
        "#009e73",
    )
    # Two days HO2Sat
    ih.plot_histogram(
        fig,
        HO2Sat,
        ho2sat_two_days,
        HO2Sat.a,
        HO2Sat.b,
        4,
        2,
        "HO2Sat" + " shared posterior",
        "#0072b2",
    )
    ih.plot_histogram(
        fig,
        HO2Sat,
        ho2sat_two_days_fef,
        HO2Sat.a,
        HO2Sat.b,
        4,
        4,
        "HO2Sat" + " shared posterior with FEF25-75",
        "#0072b2",
    )
    # Two days AR
    ih.plot_histogram(
        fig,
        AR,
        ar_day_1,
        AR.a,
        AR.b,
        5,
        1,
        "AR" + " day 1",
        colour="#d55e00",
    )
    ih.plot_histogram(
        fig,
        AR,
        ar_max,
        AR.a,
        AR.b,
        6,
        1,
        "AR" + " max",
        colour="#d55e00",
    )
    ih.plot_histogram(
        fig,
        AR,
        ar_day_1_fef,
        AR.a,
        AR.b,
        5,
        3,
        "AR" + " day 1 with FEF25-75",
        colour="#d55e00",
    )
    ih.plot_histogram(
        fig,
        AR,
        ar_max_fef,
        AR.a,
        AR.b,
        6,
        3,
        "AR" + " max with FEF25-75",
        colour="#d55e00",
    )
    # Two days IA
    ih.plot_histogram(
        fig,
        IA,
        ia_day_1,
        IA.a,
        IA.b,
        5,
        2,
        "IA" + " day 1",
        "#cc79a7",
    )
    ih.plot_histogram(
        fig,
        IA,
        ia_max,
        IA.a,
        IA.b,
        6,
        2,
        "IA" + " max",
        "#cc79a7",
    )
    ih.plot_histogram(
        fig,
        IA,
        ia_day_1_fef,
        IA.a,
        IA.b,
        5,
        4,
        "IA" + " day 1 with FEF25-75",
        "#cc79a7",
    )
    ih.plot_histogram(
        fig,
        IA,
        ia_max_fef,
        IA.a,
        IA.b,
        6,
        4,
        "IA" + " max with FEF25-75",
        "#cc79a7",
    )
    # Plot ecFEV1 profile
    fig.add_trace(
        go.Scatter(y=df["ecFEV1"], x=df["Date Recorded"], mode="markers"),
        row=7,
        col=1,
    )
    fig.update_yaxes(
        # nticks=20,
        title="ecFEV1 (L)",
        row=7,
        col=1,
    )
    fig.add_trace(
        go.Scatter(y=df["O2 Saturation"], x=df["Date Recorded"], mode="markers"),
        row=8,
        col=1,
    )
    fig.update_yaxes(
        # nticks=20,
        title="O2 saturation (%)",
        row=8,
        col=1,
    )
    # Add drug therapy
    drug_df = bd.load_drug_therapies()
    drug_df = drug_df[drug_df["DrugTherapyType"] != "Unknown"]
    drug_df_for_ID = drug_df[drug_df.ID == df_query_res_one_day.loc[0, "ID"]]

    def drug_therapy_color_dict():
        return {
            "Trikafta": "green",
            "Ivacaftor": "purple",
            "Symkevi": "purple",
            "Orkambi": "purple",
        }

    for _, row in drug_df_for_ID.iterrows():
        start_date = row.DrugTherapyStartDate
        stop_date = row.DrugTherapyStopDate
        if pd.isnull(stop_date):
            stop_date = df["Date Recorded"].max()
        fig.add_vrect(
            x0=start_date,
            # y0=0,
            x1=stop_date,
            # y1=100,
            fillcolor=drug_therapy_color_dict()[row.DrugTherapyType],
            opacity=0.08,
            layer="below",
            line_width=0,
            name=row.DrugTherapyType,
            row=7,
            col=1,
        )
        fig.add_vrect(
            x0=start_date,
            # y0=0,
            x1=stop_date,
            # y1=100,
            fillcolor=drug_therapy_color_dict()[row.DrugTherapyType],
            opacity=0.08,
            layer="below",
            line_width=0,
            name=row.DrugTherapyType,
            row=8,
            col=1,
        )
        # Add annotation
        fig.add_annotation(
            x=start_date,
            y=df['ecFEV1'].max()*1.02,
            # xref="x",
            # yref="paper",
            text=row.DrugTherapyType,
            showarrow=False,
            font=dict(size=8),
            row=7,
            col=1
        )

    # Change marker size for plot row 7
    fig.update_traces(marker=dict(size=3), selector=dict(mode="markers"))

    fig.update_xaxes(title_standoff=5)

    fig.update_layout(
        title=f"{title_prefix}<br><br>Current day:   {title_day}<br>Max FEV1 day:{title_max_FEV1_day}",
        width=1000,
        height=1000,
        font=dict(size=8),
        showlegend=False,
    )

    if save:
        fig.write_image(
            dh.get_path_to_main()
            + f"PlotsBreathe/Two days model/{title_prefix}, {title_day}.pdf"
        )
    else:
        fig.show()
    return -1

In [150]:
def infer_and_plot_for_id(df_for_ID, debug, diff_threshold=1e-8):
    df_for_ID = df_for_ID.reset_index(drop=True)
    print(f"\nID: {df_for_ID.ID.iloc[0]}")
    print(f"#datapoints: {len(df_for_ID)}")

    height = df_for_ID.Height.iloc[0]
    age = df_for_ID.Age.iloc[0]
    sex = df_for_ID.Sex.iloc[0]
    (
        _,
        inf_alg,
        HFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(height, age, sex)

    # Set variables parametrisation
    key_hfev1 = f"['{ecFEV1.name}', '{HFEV1.name}', '{AR.name}'] -> {HFEV1.name}"
    key_ho2sat = f"['{O2SatFFA.name}', '{HO2Sat.name}', '{AR.name}'] -> {HO2Sat.name}"
    HFEV1.set_factor_node_key(key_hfev1)
    HO2Sat.set_factor_node_key(key_ho2sat)

    vars = [AR, IA]
    shared_vars = [HFEV1, HO2Sat]
    obs_vars = [ecFEV1.name, O2Sat.name]
    obs_vars_fef = [ecFEV1.name, O2Sat.name, ecFEF2575prctecFEV1.name]

    # Find the max FEV1 values
    # Given an ID, get the data which maximises ecFEV1, then ecFEF2575, then O2 Saturation
    idx_max_FEV1 = df_for_ID.sort_values(
        by=["ecFEV1", "ecFEF2575", "O2 Saturation"], ascending=False
    ).index[0]

    # Randomly select 4 entries for this ID
    # Select 1 entry's index in the bottom 10 percentile of ecFEV1
    bottom_10 = (df_for_ID["ecFEV1"] == df_for_ID["ecFEV1"].quantile(0.1)).argmax()
    # Select 1 entry's index in the 50% percentile
    median = (df_for_ID["ecFEV1"] == df_for_ID["ecFEV1"].quantile(0.5)).argmax()
    # Select 1 entry's index in the top 10% percentile
    top_10 = (df_for_ID["ecFEV1"] == df_for_ID["ecFEV1"].quantile(0.8)).argmax()
    idx_list = [bottom_10, median, top_10]
    # idx_list = list(np.random.choice(df_for_ID.index, 4))

    for i, _ in df_for_ID.iloc[idx_list].iterrows():
        df_one_day = df_for_ID.iloc[[i]]
        df_two_days = df_for_ID.iloc[[i, idx_max_FEV1]]

        df_query_res_one_day, _, _ = slicing.query_across_days(
            df_one_day, inf_alg, shared_vars, vars, obs_vars, diff_threshold, debug
        )
        # reset shared vars
        for var in shared_vars:
            var.reset()
        df_query_res_one_day_fef, _, _ = slicing.query_across_days(
            df_one_day, inf_alg, shared_vars, vars, obs_vars_fef, diff_threshold, debug
        )
        for var in shared_vars:
            var.reset()
        df_query_res_two_days, _, _ = slicing.query_across_days(
            df_two_days, inf_alg, shared_vars, vars, obs_vars, diff_threshold, debug
        )
        for var in shared_vars:
            var.reset()
        df_query_res_two_days_fef, _, _ = slicing.query_across_days(
            df_two_days, inf_alg, shared_vars, vars, obs_vars_fef, diff_threshold, debug
        )
        for var in shared_vars:
            var.reset()

        plot_one_and_two_days_models_res(
            df_query_res_one_day,
            df_query_res_one_day_fef,
            df_query_res_two_days,
            df_query_res_two_days_fef,
            HFEV1,
            HO2Sat,
            AR,
            IA,
            df_for_ID,
            save=True,
        )

    return (
        df_query_res_one_day,
        df_query_res_one_day_fef,
        df_query_res_two_days,
        df_query_res_two_days_fef,
    )


# df_for_ID = df[df["ID"] == "113"]
# (
#     df_query_res_one_day,
#     df_query_res_one_day_fef,
#     df_query_res_two_days,
#     df_query_res_two_days_fef,
# ) = infer_and_plot_for_id(df_for_ID, debug=False, diff_threshold=1e-2)


df.groupby("ID").apply(
    lambda df_for_ID: infer_and_plot_for_id(df_for_ID, debug=False, diff_threshold=1e-6)
)


ID: 101
#datapoints: 1680

ID: 102
#datapoints: 263

ID: 103
#datapoints: 375

ID: 104
#datapoints: 222

ID: 105
#datapoints: 6

ID: 106
#datapoints: 335

ID: 107
#datapoints: 126

ID: 108
#datapoints: 289

ID: 109
#datapoints: 94

ID: 110
#datapoints: 8

ID: 111
#datapoints: 124

ID: 112
#datapoints: 114

ID: 113
#datapoints: 714

ID: 114
#datapoints: 133

ID: 115
#datapoints: 43

ID: 116
#datapoints: 619

ID: 117
#datapoints: 270

ID: 118
#datapoints: 108

ID: 119
#datapoints: 88

ID: 120
#datapoints: 388

ID: 121
#datapoints: 87

ID: 122
#datapoints: 257

ID: 123
#datapoints: 1128

ID: 124
#datapoints: 33

ID: 125
#datapoints: 334

ID: 126
#datapoints: 60

ID: 127
#datapoints: 165

ID: 128
#datapoints: 39

ID: 129
#datapoints: 5

ID: 130
#datapoints: 282

ID: 131
#datapoints: 29

ID: 132
#datapoints: 27

ID: 133
#datapoints: 1066

ID: 134
#datapoints: 97

ID: 135
#datapoints: 26

ID: 138
#datapoints: 195

ID: 139
#datapoints: 276

ID: 140
#datapoints: 116

ID: 141
#datapoints: 201


ID
101    ([ID], [ID], [ID, Day], [ID, Day])
102    ([ID], [ID], [ID, Day], [ID, Day])
103    ([ID], [ID], [ID, Day], [ID, Day])
104    ([ID], [ID], [ID, Day], [ID, Day])
105    ([ID], [ID], [ID, Day], [ID, Day])
                      ...                
549    ([ID], [ID], [ID, Day], [ID, Day])
550    ([ID], [ID], [ID, Day], [ID, Day])
552    ([ID], [ID], [ID, Day], [ID, Day])
553    ([ID], [ID], [ID, Day], [ID, Day])
554    ([ID], [ID], [ID, Day], [ID, Day])
Length: 353, dtype: object

In [143]:
# Select index for the quantile
df_for_ID = df_for_ID.reset_index(drop=True)
idx = (df_for_ID["ecFEV1"] == df_for_ID["ecFEV1"].quantile(0.9)).argmax()

1.76
108
375


ID                                    103
Date Recorded                  2020-09-03
FEV1                                 1.75
O2 Saturation                          97
FEF2575                              0.84
ecFEV1                               1.76
ecFEF2575                            0.84
Sex                                Female
Height                              161.0
Age                                    39
Predicted FEV1                   2.979444
Healthy O2 Saturation             98.1863
ecFEV1 % Predicted              59.071426
FEV1 % Predicted                58.735793
O2 Saturation % Healthy         98.791787
ecFEF2575%ecFEV1                     48.0
Max ecFEV1                           1.82
Max ecFEF2575                        0.86
idx ecFEV1 (L)                         35
idx ecFEF25-75 % ecFEV1 (%)            24
idx O2 saturation (%)                  47
Name: 108, dtype: object

In [146]:
pd.Series([False, True, True, False]).argmax()

1

In [95]:
(
    _,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    ecFEF2575prctecFEV1,
) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(170, 30, "Female")

plot_one_and_two_days_models_res(
    df_query_res_one_day,
    df_query_res_one_day_fef,
    df_query_res_two_days,
    df_query_res_two_days_fef,
    HFEV1,
    HO2Sat,
    AR,
    IA,
    df_for_ID,
)

-1

In [ ]:
print(
    f"HFEV1 max diff between 2 days: {max(df_query_res_two_days.loc[0, HFEV1.name] - df_query_res_two_days.loc[1, HFEV1.name]):.2g}"
)
print(
    f"HO2Sat max diff between 2 days: {max(df_query_res_two_days.loc[0, HO2Sat.name] - df_query_res_two_days.loc[1, HO2Sat.name]):.2g}"
)
# Stop criteria: difference of the last HFEV1 inferred between two epochs is les than a threshold (1e-6)

In [48]:
# Make query across days function with one day gives the same results as a one day inference -> yes
(
    _,
    inf_alg,
    HFEV1,
    ecFEV1,
    AR,
    HO2Sat,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    ecFEF2575prctecFEV1,
) = mb.o2sat_fev1_fef2575_point_in_time_model_shared_healthy_vars(160, 40, "Female")

df_one_day = df_for_ID.iloc[[0]]
df_query_res, df_res_before_convergence, shared_vars_final = slicing.query_across_days(
    df_one_day, inf_alg, shared_vars, vars, obs_vars, 1e-8
)

res = ih.infer_on_factor_graph(
    inf_alg,
    [HFEV1, HO2Sat, AR, IA],
    [[ecFEV1, df_one_day.ecFEV1[0]], [O2Sat, df_one_day["O2 Saturation"][0]]],
)

print("Are the results the same?")
print(df_query_res[AR.name][0] - res[AR.name].values)
print(df_query_res[IA.name][0] - res[IA.name].values)

NameError: name 'shared_vars' is not defined

# Use exact inference

In [ ]:
import src.data.breathe_data as bd
import src.inference.long_inf_slicing as slicing
import src.models.builders as mb
import src.models.var_builders as var_builders
import src.inference.helpers as ih
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np

df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_with_idx_and_heighest_obs_per_id")
df_for_ID = df[df["ID"] == "101"]
idx_max_FEV1 = df_for_ID.sort_values(
    by=["ecFEV1", "ecFEF2575", "O2 Saturation"], ascending=False
).index[0]

df_for_ID.iloc[[0, idx_max_FEV1]]

In [ ]:
height = df_for_ID.Height.iloc[0]
age = df_for_ID.Age.iloc[0]
sex = df_for_ID.Sex.iloc[0]
(
    model,
    inf_alg,
    HFEV1,
    HO2Sat,
    ecFEV1,
    AR,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    # ecFEF2575prctecFEV1,
    ecFEV1_2,
    AR_2,
    O2SatFFA_2,
    IA_2,
    UO2Sat_2,
    O2Sat_2,
    # ecFEF2575prctecFEV1_2,
) = mb.o2_sat_fev1_fef2575_two_days_model(height, age, sex)

In [ ]:
# vars = [HFEV1, HO2Sat, AR, IA, AR_2, IA_2]
vars = [AR]
obs_vars = [[ecFEV1, 1.31], [O2Sat, 97], [ecFEV1_2, 1.79], [O2Sat_2, 98]]

res = ih.infer(inference_alg=inf_alg, variables=vars, evidences=obs_vars)

In [ ]:
res